# Save IMG

In [ ]:
QUANTILE = 0.4

postfix = f"_{str(QUANTILE).replace(".", "")}"

In [ ]:
import pandas as pd
import os

base_dir = 'data'

data = pd.read_csv("train.csv")

file_names = {}

quantiles = data.iloc[:, 1:].apply(lambda row: row[row != 0].quantile(QUANTILE), axis=1)

mask = data.iloc[:, 1:].gt(quantiles, axis=0) & (data.iloc[:, 1:] != 0)

data.iloc[:, 1:] = mask.astype(int)

for i in range(10):
    os.makedirs(os.path.join(base_dir, str(i)), exist_ok=True)

def process_row(row):    
    n_of_1 = row.drop('label').values.sum()
    if n_of_1 < 35 or n_of_1 > 100:
        return
    
    label = row['label']

    data = ''.join(map(str, row.drop('label').values))
    folder_path = os.path.join(base_dir, str(label))

    if label in file_names:
        file_names[label] += 1
    else:
        file_names.update({label: 0})
    
    file_name = os.path.join(folder_path, f"{file_names[label]}{postfix}.txt")
    
    with open(file_name, 'w') as file:
        file.write(data)

data.apply(process_row, axis=1)

# Load IMG

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

number = 7

for i in range(0, 20):
    path_to_img = f"data/{number}/{i}{postfix}.txt"
    with open(path_to_img, 'r') as file:
        file_content = file.readline()

    data_list = [int(char) for char in file_content.strip()]
    data_array = np.array(data_list).reshape(28, 28)

    plt.imshow(data_array, cmap='gray_r', interpolation='nearest')
    plt.axis('off')
    plt.show()

# Pixel Distribution

In [ ]:
import plotly.express as px
import pandas as pd

data = pd.read_csv("train.csv")

pixel_values = data.drop(columns=['label']).values.flatten()
non_zero_pixel_values = pixel_values[pixel_values != 0]

fig = px.histogram(
    non_zero_pixel_values,
    nbins=256,
    range_x=[0, 255],
    title='Distribution of Pixel Values Across All Columns',
    labels={'value': 'Pixel Value', 'count': 'Frequency'},
    opacity=0.7
)

fig.update_layout(
    xaxis_title='Pixel Value',
    yaxis_title='Frequency',
    bargap=0.1
)

fig.show()